<a href="https://colab.research.google.com/github/Vinayak-Sharma12/LangGraph/blob/main/9_Router_Made_using_Function(Pydantic).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q langgraph langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 17.5 MB/s eta 0:00:00


DEFINE LLM

In [5]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant',api_key='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')

## **CREATE STATE**

In [7]:
from typing import TypedDict,Literal
class State(TypedDict):
  input:str
  decision:str
  output:str

## **CREATE ROUTER FUNCTION**

WORKFLOW -> DEFINE PYDANTIC(To Get Structure Output) -> CREATE A NODE TO DECISION MAKING -> CREATE CONDITIONAL FUNCTION TO ROUTE THAT

In [25]:
from pydantic import BaseModel,Field
class route(BaseModel):
  step:Literal['story','poem','joke']

In [23]:
router=llm.with_structured_output(route)

In [42]:
def route_node(state:State):
  """Route to the appropriate Node"""
  return {"decision":router.invoke(f"According to user demand {state['input']} Route to either story,poem or joke ").step}

In [44]:
def route_fxn(state:State):
  if state['decision']=='story':
    return "story"
  if state['decision']=='poem':
    return "poem"
  if state['decision']=='joke':
    return "joke"

## **CREATE NODE**

In [26]:
def story(state:State):
  """Write a story"""
  return {"output":llm.invoke(f"Write a story on the {state['input']}").content}

In [27]:
def poem(state:State):
  """Write a poem"""
  return {"output":llm.invoke(f"Write a poem on the {state['input']}").content}


In [28]:
def joke(state:State):
  """Write a joke"""
  return {"output":llm.invoke(f"Write a joke on the {state['input']}").content}

## **GRAPH**

In [47]:
from langgraph.graph import StateGraph,START,END

#DEFINE STATEGRAPH
builder=StateGraph(State)

#Add Nodes
builder.add_node("story",story)
builder.add_node("poem",poem)
builder.add_node("joke",joke)
builder.add_node("route_node",route_node)

#Add Edges
builder.add_edge(START,"route_node")
builder.add_conditional_edges("route_node",route_fxn,{
    "story":"story",
    "poem":'poem',
    "joke":"joke"
})
builder.add_edge("story",END)
builder.add_edge("poem",END)
builder.add_edge("joke",END)

#Compile
graph=builder.compile()

In [58]:
result=graph.invoke({"input":"Write a poem on Elon Musk"})

In [59]:
print(result['output'])

In Silicon Valley's land of dreams,
A visionary rose to great extremes,
Elon Musk, a name that's known so well,
A pioneer, a trailblazer, a story to tell.

With SpaceX, he reached for the skies,
A dream of Mars, a future surprise,
Reusability, efficiency, a goal in sight,
To make humanity a multi-planetary sight.

At Tesla, he revolutionized the wheel,
Electric cars, a new era to feel,
Sustainable energy, a path to take,
A cleaner future, for the world's sake.

With Neuralink, he sought to make us whole,
A brain-machine interface, a new goal,
To merge humans and tech, a new way to be,
A future of possibilities, for you and me.

At The Boring Company, he dug deep,
To make tunnels, and transportation unique,
A solution to traffic, a problem to solve,
A faster future, for the world to evolve.

With Twitter, he shared his thoughts so bold,
A platform for discussion, a story to be told,
A voice for the people, a place to share,
A digital stage, for the world to care.

Elon Musk, a name that